<a href="https://colab.research.google.com/github/dohi1004/AI-teamProject-brailleTranslation/blob/main/AI_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# **AI Team Project Final Presentation**

### **Team 6**

16102269 Kim Jong Gyu

19102095 Lee Do Hui
***



# **1. Motivation For The Proeject**
* Similar to general language, braille is different according to the world.
* T

시각 장애인이 외국어로 되어 있는 점자를 읽어야 하는 경우 해당 나라 언어의 점자를 배운 적이 없다면 읽을 수 없습니다. 따라서, 외국어 점자를 모국어 음성으로 변환해주는 application을 만들고자 합니다.

# **2. Problem Description**
Here, we suppose the use case as follows. Here, we use similar word order languages (English, French) because of using basic NMT model.

1) English blind 

2) French

1. English blind write braille 
2. Translate English braille to English (with Braille Translation)
3. Translate English to French (with Neural Machine Translation)


**Braille Translation and Language Translation**

### 1) Braille Translation

Translate English braille to English.

### 2) Language Translation

Translate English to French.

**Finally, our application can translate other language's braille to target person's language by connecting those two models.** 



# **3. Data Description**

### **Braille Translation**
*   ~~Preferentially, I use braille notation in English dataset in Kaggle~~
  *   ~~Reference : https://www.kaggle.com/code/kwisatzhaderach/braille-classifier-keras/data~~

### **Neural Machine Translation**

*   For neural machine translation model, we used dataset comprised of French phrases and their English counterparts. The dataset is available from the http://www.manythings.org/anki/, with examples drawn from the Tatoeba Project.

<img src = "https://drive.google.com/uc?id=1zgHVKVLXsjyD4re6bWE5baV0q71J4yQo" height = 250 width = 500>

* The fra.txt file contains pairs of English to French phrases, one pair per line with a tab separating the language. Total pairs of sentences are 197,463.

<img src = "https://drive.google.com/uc?id=1-WO41TvUqFickYyjN298293hUtBSZRUs" height = 150 width = 600>

------------------
Preprocessing process of each data is explained in 5. Implementation Details

------------------

# **4.Model Architecture**

### 1) Model 1: Convolutional Neural Network

a model consisting of ~~~~> 

### 2) Model 2: Seq2Seq model  - LSTM & GRU

Encoder-decoder based model. By comparing the performance of LSTM and GRU model each, we decide to use ~~ model. 

# **5. Implementation Details**

## **Part1. Braille Translation Model** 

### **Hyperparameter Selection Process**

##  **Part2. NMT model (English -> French)**


In [ ]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **Data and preprocessing**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/') 

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
data = pd.read_csv("./gdrive/MyDrive/AI/teamProject/fra.txt", delimiter = "\t")
data.columns = ["en", "fr", "cc"]
data

,en,fr,cc
0,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
197457,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
197458,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
197459,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
197460,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [ ]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
def to_ascii(s):
    # delete accent in French
    # EX) : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
    # Call accent deletion function
    sent = to_ascii(sent.lower())
    
    # Add blank between word and punctuation 
    # ex) "I am a student." => "I am a student ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)
    
    # Except (a-z, A-Z, ".", "?", "!", ","), transform others to blank
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    
    # Transform multiple blanks to one blank
    sent = re.sub(r"\s+", " ", sent)
    
    return sent

In [ ]:
# Preprocessing Test
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('Before preprocessing English sentence :', en_sent)
print('After preprocessing English sentence :',preprocess_sentence(en_sent))
print('Before preprocessing French sentence :', fr_sent)
print('After preprocessing French sentence :', preprocess_sentence(fr_sent))

Before preprocessing English sentence : Have you had dinner?
After preprocessing English sentence : have you had dinner ?
Before preprocessing French sentence : Avez-vous déjà diné?
After preprocessing French sentence : avez vous deja dine ?


In [ ]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    src_seqlen = 0
    tar_seqlen = 0
    with open("./gdrive/MyDrive/AI/teamProject/fra.txt", "r", encoding='UTF-8') as lines:
        for i, line in enumerate(lines):
            # split source data and target data 
            src_line, tar_line, _ = line.strip().split('\t')
            
            # preprocess source data
            preprocessed_sen = preprocess_sentence(src_line)
            src_line = [w for w in preprocessed_sen.split()]
            for w in preprocessed_sen.split():
              if len(w) > src_seqlen:
                src_seqlen = len(w)
            
            # preprocess target data
            tar_line = preprocess_sentence(tar_line)
            for w in tar_line.split():
              if len(w) > tar_seqlen:
                tar_seqlen = len(w)
                
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]
            
            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)
            
            if i == num_samples - 1:
                break
        
    return encoder_input, decoder_input, decoder_target, src_seqlen, tar_seqlen

In [ ]:
length = 0
with open("./gdrive/MyDrive/AI/teamProject/fra.txt", "r", encoding='UTF-8') as lines:
        for i, line in enumerate(lines):
            length+=1
print(length)

197463


In [ ]:
num_samples = 70000

In [ ]:
sents_en_in, sents_fra_in, sents_fra_out, src_seqlen, tar_seqlen = load_preprocessed_data()
print('Input of Encoder :',sents_en_in[:5])
print('Input of Decoder :',sents_fra_in[:5])
print('Label of Decoder :',sents_fra_out[:5])
print(src_seqlen)
print(tar_seqlen)

Input of Encoder : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
Input of Decoder : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
Label of Decoder : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]
16
18


In [ ]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [ ]:
print('Input of Encoder Shape :',encoder_input.shape)
print('Input of Decoder Shape :',decoder_input.shape)
print('Label of Decoder Shape :',decoder_target.shape)

Input of Encoder Shape : (70000, 9)
Input of Decoder Shape : (70000, 17)
Label of Decoder Shape : (70000, 17)


In [ ]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("Size of English word set : {:d}, Size of French word set : {:d}".format(src_vocab_size, tar_vocab_size))

Size of English word set : 7241, Size of French word set : 12336


In [ ]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('Random Sequence :',indices)

Random Sequence : [26140 32868 26191 ... 35678 11235 25854]


In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [ ]:
print(encoder_input[30997])
print(decoder_input[30997])
print(decoder_target[30997])

[  19    7 2637    1    0    0    0    0    0]
[  2  24   6 286   1   0   0   0   0   0   0   0   0   0   0   0   0]
[ 24   6 286   1   3   0   0   0   0   0   0   0   0   0   0   0   0]


In [ ]:
n_of_val = int(num_samples*0.1)
print('Number of validation data :',n_of_val)

Number of validation data : 7000


In [ ]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
print('Shape of training source data :',encoder_input_train.shape)
print('Shape of training target data :',decoder_input_train.shape)
print('Shape of trainig target label data :',decoder_target_train.shape)
print('Shape of test source data :',encoder_input_test.shape)
print('Shape of test target data :',decoder_input_test.shape)
print('Shape of test target label data :',decoder_target_test.shape)

Shape of training source data : (63000, 9)
Shape of training target data : (63000, 17)
Shape of trainig target label data : (63000, 17)
Shape of test source data : (7000, 9)
Shape of test target data : (7000, 17)
Shape of test target label data : (7000, 17)


## **LSTM model**

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

#### **Hyperparameter Tuning**

In [ ]:
def LSTM_Model(embedding_dim, hidden_units, lr, b1, b2, batchsize, encoder_dropout, decoder_dropout):
  encoder_inputs = Input(shape=(None,))
  enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # Embedding layer
  enc_masking = Masking(mask_value=0.0)(enc_emb) # Exclude padding 0 in operation
  encoder_lstm = LSTM(hidden_units, return_state=True, dropout=encoder_dropout) # To return state value
  encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # Return hidden state and cell state
  encoder_states = [state_h, state_c] # Save encoder's hidden state and cell state

  # Decoder
  decoder_inputs = Input(shape=(None,))
  dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # Embedding layer
  dec_emb = dec_emb_layer(decoder_inputs) # exclude padding 0 in operation
  dec_masking = Masking(mask_value=0.0)(dec_emb)

  # To return state value, return_state = True
  # To predict word for every time step, return_sequences = True
  decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True, dropout=decoder_dropout) 

  # Use encoder's hidden state as initial hidden state 
  decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                      initial_state=encoder_states)

  # predict word bsaed on softmax activation function for all results from every time step
  decoder_dense = Dense(tar_vocab_size, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)

  # Model's input and output 
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(optimizer=optimizers.Adam(learning_rate=lr, beta_1=b1, beta_2=b2), loss='sparse_categorical_crossentropy', metrics=['acc'])

  history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=batchsize, callbacks=[EarlyStopping(monitor='val_loss', patience = 3)], epochs=10) # for testing, we use epochs = 10 

  return history.history['val_loss'][-1]

In [ ]:
def get_hyper_param(n_iteration):
  hyper_param = []  # learning_rate, n_hidden, timestep, epochs
  for i in range(n_iteration):
    current_params = []
    # We use adam optimizer, so we tune learning rate
    current_params.append(np.random.uniform(0,0.1)) # learning rate 
    current_params.append(np.random.uniform(0.5,0.999)) # b1 (from Momentum)
    current_params.append(np.random.uniform(0.5,0.999)) # b2 (from RMSProp)
    current_params.append(np.random.randint(1,513)) # hidden units 
    current_params.append(np.random.randint(32,513)) # batch_size
    current_params.append(np.random.randint(50,300)) # embedding dimension
    current_params.append(np.random.uniform(0,0.5)) # encoder dropout
    current_params.append(np.random.uniform(0,0.5)) # decoder dropout
    hyper_param.append(current_params)
  return hyper_param

hyper_parameter = get_hyper_param(30)
train_loss, val_loss = list(), list()
best_params = []

min_val_loss = 999
for alpha, b1, b2, hidden_units, batch_size, embedding_dim, en_dropout, de_dropout in hyper_parameter:
  print('lr',alpha, 'b1', b1, 'b2', b2, 'n_hiddens', hidden_units,'batch_size', batch_size, 'embedding_dim', embedding_dim, 
        'en_dropout',  en_dropout, 'de_dropout', de_dropout)

  # model 정의 (model.add ~ model.compile 까지)
  current_val_loss = LSTM_Model(embedding_dim, hidden_units, alpha, b1, b2, batch_size, en_dropout, de_dropout)

  if current_val_loss < min_val_loss:
    min_val_loss = current_val_loss
    best_params = [alpha, b1, b2, hidden_units, batch_size, embedding_dim, en_dropout, de_dropout]
    print('best_params',best_params)
  
print('final best params',best_params)

lr 0.06270461188149924 b1 0.843482643498749 b2 0.8925307649479214 n_hiddens 376 batch_size 455 embedding_dim 225 en_dropout 0.25149046826372107 de_dropout 0.28754374782314557
Epoch 1/10
139/139 [==============================] - 30s 161ms/step - loss: 1.9090 - acc: 0.7286 - val_loss: 1.3475 - val_acc: 0.7799
Epoch 2/10
139/139 [==============================] - 19s 140ms/step - loss: 1.2717 - acc: 0.7855 - val_loss: 1.2778 - val_acc: 0.7887
Epoch 3/10
139/139 [==============================] - 20s 141ms/step - loss: 1.2140 - acc: 0.7914 - val_loss: 1.2685 - val_acc: 0.7928
Epoch 4/10
139/139 [==============================] - 19s 140ms/step - loss: 1.1871 - acc: 0.7942 - val_loss: 1.2826 - val_acc: 0.7924
Epoch 5/10
139/139 [==============================] - 19s 139ms/step - loss: 1.1731 - acc: 0.7954 - val_loss: 1.2741 - val_acc: 0.7956
Epoch 6/10
139/139 [==============================] - 19s 139ms/step - loss: 1.1693 - acc: 0.7960 - val_loss: 1.2842 - val_acc: 0.7953
best_params [0.

# **Final LSTM model**

Based on best hyperparameter from hyperparameter tuning process, we build our final LSTM model with them. 

In [ ]:
# learning rate, b1, b2, hidden_units, batch_size, embedding_dim, encoder dropout, decoder dropout
print('final best params',best_params)
LSTM_final_lr = best_params[0]
LSTM_final_b1 = best_params[1]
LSTM_final_b2 = best_params[2]
hidden_units = best_params[3]
LSTM_final_batch = best_params[4]
embedding_dim = best_params[5]
LSTM_final_enD = best_params[6]
LSTM_final_deD = best_params[7]

final best params [0.015886169959430787, 0.5930491086706786, 0.9364600597831465, 312, 273, 166, 0.1266038983112921, 0.47534027805897144]


## **LSTM evaluation**

In NMT, there are diverse evaluation metrics from human evaluation methods to automatic evaluation metrics [??]. Here, we use some automatic evaluation metrics for evaluating our nmt model's performance.

**For Lexical Similarity**

* BLEU Score

BLEU is based on the degree of n-gram overlapping between the strings of words produced by the machine and the human translation references at the corpus level. BLEU computes the precision for n-gram of size 1-to-4 with
the coefficient of brevity penalty (BP).

* ROUGE Score

* Mete

**For Linguistic Features**

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # Embedding layer
enc_masking = Masking(mask_value=0.0)(enc_emb) # Exclude padding 0 in operation
encoder_lstm = LSTM(hidden_units, return_state=True) # To return state value
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # Return hidden state and cell state
encoder_states = [state_h, state_c] # Save encoder's hidden state and cell state

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # Embedding layer
dec_emb = dec_emb_layer(decoder_inputs) # exclude padding 0 in operation
dec_masking = Masking(mask_value=0.0)(dec_emb)

# To return state value, return_state = True
# To predict word for every time step, return_sequences = True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# Use encoder's hidden state as initial hidden state 
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# predict word bsaed on softmax activation function for all results from every time step
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model's input and output 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=optimizers.Adam(learning_rate=LSTM_final_lr, beta_1=LSTM_final_b1, beta_2=LSTM_final_b2), 
              loss='sparse_categorical_crossentropy', metrics=['acc'])

model_weights_path = './gdrive/MyDrive/AI/teamProject/LSTM/weights/weights.ckpt'

save_best_weights = ModelCheckpoint(
  model_weights_path, monitor='val_loss', mode='min',
  save_weights_only=True, save_best_only=True, verbose=1, 
)

history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
        validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
        batch_size=LSTM_final_batch, callbacks=[save_best_weights,EarlyStopping(monitor='val_loss', patience = 10)], epochs=50) 

Epoch 1/50
231/231 [==============================] - ETA: 0s - loss: 1.4450 - acc: 0.7705
Epoch 1: val_loss improved from inf to 0.95634, saving model to ./gdrive/MyDrive/AI/teamProject/LSTM/weights/weights.ckpt
231/231 [==============================] - 30s 93ms/step - loss: 1.4450 - acc: 0.7705 - val_loss: 0.9563 - val_acc: 0.8297
Epoch 2/50
231/231 [==============================] - ETA: 0s - loss: 0.8016 - acc: 0.8484
Epoch 2: val_loss improved from 0.95634 to 0.77351, saving model to ./gdrive/MyDrive/AI/teamProject/LSTM/weights/weights.ckpt
231/231 [==============================] - 20s 86ms/step - loss: 0.8016 - acc: 0.8484 - val_loss: 0.7735 - val_acc: 0.8564
Epoch 3/50
231/231 [==============================] - ETA: 0s - loss: 0.6150 - acc: 0.8735
Epoch 3: val_loss improved from 0.77351 to 0.69534, saving model to ./gdrive/MyDrive/AI/teamProject/LSTM/weights/weights.ckpt
231/231 [==============================] - 20s 86ms/step - loss: 0.6150 - acc: 0.8735 - val_loss: 0.6953 - 

In [ ]:
# Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder 
# Tensor for previous step's state 
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Reuse Embedding layer that was used in training phase 
dec_emb2 = dec_emb_layer(decoder_inputs)

# To predict next word.. 
# Previous time step's state -> current time step's initial state  
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# Predict word in every time step 
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Updated decoder 
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    states_value = encoder_model.predict(input_seq)
    
    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']
    
    stop_condition = False
    decoded_sentence = ''
    
    # Iterate loop until stop_condition becomes True
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char
        
        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True
            
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
        
    return decoded_sentence

In [ ]:
# transform original sentence's integer sequence to text sequence 
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence

# transform translated sentence's integer sequence to text sequence
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
            
    return sentence

In [ ]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    
    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

[[ 21   3  33 136   4   0   0   0   0]]
1/1 [==============================] - 0s 16ms/step
입력문장 : are you all ready ? 
정답문장 : etes vous tous prets ? 
번역문장 : etes vous toutes dingues ? 
--------------------------------------------------
[[  2 634 336 280 305   1   0   0   0]]
1/1 [==============================] - 0s 17ms/step
입력문장 : i drank beer last night . 
정답문장 : j ai bu de la biere la nuit derniere . 
번역문장 : j ai la biere pour la biere . 
--------------------------------------------------
[[   8    9    5 2103  157    1    0    0    0]]
1/1 [==============================] - 0s 19ms/step
입력문장 : it s a sunny day . 
정답문장 : c est un jour ensoleille . 
번역문장 : c est une journee journee . 
--------------------------------------------------
[[  2  17  92  12 544   1   0   0   0]]
1/1 [==============================] - 0s 18ms/step
입력문장 : i m going to fight . 
정답문장 : je vais me battre . 
번역문장 : je vais etre prudente . 
--------------------------------------------------
[[   3   13 1931   

일부 환경에서 완전한 입력-목표 시퀀스 쌍을 버퍼링할 수 없듯이(예를 들어, 만약 매우 긴 시퀀스는 online 학습) 이는 전체 목표 시퀀스로 접근할 수 없기 때문에 Teacher forcing을 사용할 수 없습니다. 이 경우 decoder의 예측값을 입력으로 재입력하여 학습을 실행할 수 있습니다. (그저 추론될 수 있도록)

출력값을 재주입하는 루프를 설계한 모델을 구축하면 다음과 같은 결과를 얻을 수 있습니다.

위와 같이 입력을 Ground Truth로 넣어주게 되면,

학습시 더 정확한 예측이 가능하게 되기 때문에

초기 학습 속도를 빠르게 올릴 수 있다.

### **GRU model**

#### **Hyperparameter Selection Process**

With Random Search

In [ ]:
from tensorflow.keras.layers import Input, Embedding, GRU
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [ ]:
def create_train_model(
  enc_seqlen, dec_seqlen, enc_vocabsize, dec_vocabsize, hsize, 
  embsize, encoder_dropout, decoder_dropout):
  ### Encoder
  encoder_inputs = Input(shape=(None,), name="encoder_input")
  embedding = Embedding(
      enc_vocabsize, embsize, name="encoder_embedding"
  )
  encoder_gru = GRU(
      hsize, return_state=True, name="encoder_gru", dropout=encoder_dropout
  )
  encoder_emb = embedding(encoder_inputs)
  encoder_outputs, encoder_state = encoder_gru(encoder_emb)

  ### Decoder
  decoder_inputs = Input(shape=(None,), name="decoder_input")
  embedding = Embedding(
      dec_vocabsize, embsize, input_length=dec_seqlen-1, name="decoder_embedding"
   )
  decoder_gru = GRU(
      hsize, return_state=True, return_sequences=True, name="decoder_gru", 
      dropout=decoder_dropout
  )

  decoder_emb = embedding(decoder_inputs)
  decoder_outputs, _ = decoder_gru(decoder_emb, initial_state=encoder_state)
  dense_layer = Dense(dec_vocabsize, activation="softmax", name="dense_layer")
  decoder_outputs = dense_layer(decoder_outputs)

  # Define the Model which accepts encoder/decoder inputs and outputs predictions
  model = Model(
      inputs=[encoder_inputs, decoder_inputs],
      outputs=decoder_outputs,
      name="encoder_decoder_model_gru",
  )
  return model

In [ ]:
def GRU_Model(embedding_dim, hidden_units, lr, b1, b2, batchsize, encoder_dropout, decoder_dropout):
  model = create_train_model(
    src_seqlen, tar_seqlen, src_vocab_size, tar_vocab_size, 
    hidden_units, embedding_dim, 
    encoder_dropout, decoder_dropout
  )

  model.compile(optimizer=optimizers.Adam(learning_rate=lr, beta_1=b1, beta_2=b2), loss='sparse_categorical_crossentropy', metrics=['acc'])

  history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=batchsize, callbacks=[EarlyStopping(monitor='val_loss', patience = 3)], epochs=10) # for testing, we use epochs = 10 


  return history.history['val_loss'][-1]

In [ ]:
def get_hyper_param(n_iteration):
  hyper_param = []  # learning_rate, n_hidden, timestep, epochs
  for i in range(n_iteration):
    current_params = []
    # We use adam optimizer, so we tune learning rate
    current_params.append(np.random.uniform(0,0.1)) # learning rate 
    current_params.append(np.random.uniform(0.5,0.999)) # b1 (from Momentum)
    current_params.append(np.random.uniform(0.5,0.999)) # b2 (from RMSProp)
    current_params.append(np.random.randint(1,513)) # hidden units 
    current_params.append(np.random.randint(32,513)) # batch_size
    current_params.append(np.random.randint(50,300)) # embedding dimension
    current_params.append(np.random.uniform(0,0.5)) # encoder dropout
    current_params.append(np.random.uniform(0,0.5)) # decoder dropout
    hyper_param.append(current_params)
  return hyper_param

hyper_parameter = get_hyper_param(30)
train_loss, val_loss = list(), list()
best_params_gru = []

min_val_loss = 999
for alpha, b1, b2, hidden_units, batch_size, embedding_dim, en_dropout, de_dropout in hyper_parameter:
  print('lr',alpha, 'b1', b1, 'b2', b2, 'n_hiddens', hidden_units,'batch_size', batch_size, 'embedding_dim', embedding_dim, 
        'en_dropout',  en_dropout, 'de_dropout', de_dropout)

  # model 정의 (model.add ~ model.compile 까지)
  current_val_loss = GRU_Model(embedding_dim, hidden_units, alpha, b1, b2, batch_size, en_dropout, de_dropout)

  if current_val_loss < min_val_loss:
    min_val_loss = current_val_loss
    best_params = [alpha, b1, b2, hidden_units, batch_size, embedding_dim, en_dropout, de_dropout]
    print('best_params',best_params_gru)
  
print('final best params',best_params_gru)

lr 0.08282658244015961 b1 0.571573182981131 b2 0.8968709386260224 n_hiddens 394 batch_size 118 embedding_dim 211 en_dropout 0.37457963135095734 de_dropout 0.4368113816473733
Epoch 1/10
534/534 [==============================] - 29s 48ms/step - loss: 7.5421 - acc: 0.7012 - val_loss: 8.3749 - val_acc: 0.7068
Epoch 2/10
534/534 [==============================] - 25s 47ms/step - loss: 8.8152 - acc: 0.7130 - val_loss: 9.1793 - val_acc: 0.7228
Epoch 3/10
534/534 [==============================] - 26s 49ms/step - loss: 9.5562 - acc: 0.7138 - val_loss: 9.9835 - val_acc: 0.7214
Epoch 4/10
534/534 [==============================] - 26s 48ms/step - loss: 10.0048 - acc: 0.7136 - val_loss: 10.6554 - val_acc: 0.7164
best_params []
lr 0.08309691041604804 b1 0.6028145592359843 b2 0.6755410399211511 n_hiddens 226 batch_size 134 embedding_dim 199 en_dropout 0.42543102589446546 de_dropout 0.12908943716723353
Epoch 1/10
471/471 [==============================] - 24s 44ms/step - loss: 2.4718 - acc: 0.7476 

KeyboardInterrupt: ignored

### **Final GRU model**

In [ ]:
model_weights_path = './gdrive/MyDrive/AI/teamProject/GRU/weights/weights.ckpt'

In [ ]:
save_best_weights = tf.keras.callbacks.ModelCheckpoint(
  model_weights_path, monitor='val_loss', mode='min',
  save_weights_only=True, save_best_only=True, verbose=1, 
)

early_stopping = tf.keras.callbacks.EarlyStopping(
  monitor='val_loss', patience=2, verbose=1,
  mode='min', restore_best_weights=True, 
)

In [ ]:
def create_inference_encoder_from(input_layer, embedding_layer, gru_layer):
  encoder_embedding = embedding_layer(input_layer)
  encoder_outputs, encoder_state = gru_layer(encoder_embedding)
  encoder = Model(input_layer, encoder_state)
  return encoder

def create_inference_decoder_from(input_layer, embedding_layer, gru_layer, dense_layer):
  # decoder_input = Input(shape=(1,))
  decoder_embedding = embedding_layer(input_layer)
  input_shape = dense_layer.input.shape[-1]
  decoder_inputs_state = Input(shape=(input_shape,))

  decoder_output, decoder_output_state = gru_layer(
    decoder_embedding, initial_state=decoder_inputs_state
  )

  decoder_prediction = dense_layer(decoder_output)
  decoder = Model(
    inputs=[input_layer, decoder_inputs_state], 
    outputs=[decoder_prediction, decoder_output_state]
  )
  return decoder

In [ ]:
encoder = create_inference_encoder_from(
  model.get_layer('encoder_input').input,
  model.get_layer('encoder_embedding'),
  model.get_layer('encoder_gru')
)

decoder = create_inference_decoder_from(
  model.get_layer('decoder_input').input,
  model.get_layer('decoder_embedding'),
  model.get_layer('decoder_gru'),
  model.get_layer('dense_layer'),
)

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    state = encoder.predict(input_seq)
    
    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']
    
    stop_condition = False
    decoded_sentence = ''
    
    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, state = decoder.predict([target_seq, state])
        
        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char
        
        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True
            
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        # # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        state = [state]
        
    return decoded_sentence

## **Predicted Result & Evaluation**

### Evaluation Metrics
We use same evaluation metrics in LSTM

In [ ]:
import nltk.translate.bleu_score as bleu
from rouge import Rouge

In [ ]:
rouge = Rouge()

for seq_index in [3, 50, 100, 300, 1001]:
    reference = [[]]
    candidate = []
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    

    input_sentence = seq_to_src(encoder_input_train[seq_index])
    answer_sentence = seq_to_tar(decoder_input_train[seq_index])
    translated_sentence = decoded_sentence[1:-5]

    print("Input Sentence :", input_sentence)
    print("Answer Sentence: ", answer_sentence)
    print("Translated Sentence :", translated_sentence)

    print(rouge.get_scores([translated_sentence], [answer_sentence], avg=True))
    reference[0].extend(answer_sentence.split())
    candidate.extend(translated_sentence.split())
    print('BLEU score: {:.5f}'.format(bleu.sentence_bleu(reference, candidate, weights=(1.0, 0, 0, 0))))
    
    print("-"*50)

1/1 [==============================] - 0s 17ms/step
Input Sentence : i like busy places . 
Answer Sentence:  j aime les endroits debordant d activite . 
Translated Sentence : j aime les endroits debordant d activite . 
{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}
BLEU score: 1.00000
--------------------------------------------------
1/1 [==============================] - 0s 15ms/step
Input Sentence : i m always careful . 
Answer Sentence:  je suis toujours prudent . 
Translated Sentence : je suis toujours prudent . 
{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}
BLEU score: 1.00000
--------------------------------------------------
1/1 [==============================] - 0s 22ms/step
Input Sentence : you re a filthy liar ! 
Answer Sentence:  vous etes une sale menteuse ! 
Tr

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

1/1 [==============================] - 0s 17ms/step
Input Sentence : no one seems to know . 
Answer Sentence:  personne ne semble savoir . 
Translated Sentence : personne ne semble savoir . 
{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}
BLEU score: 1.00000
--------------------------------------------------
1/1 [==============================] - 0s 18ms/step
Input Sentence : it s quiet . 
Answer Sentence:  c est calme . 
Translated Sentence : c est calme . 
{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}
BLEU score: 1.00000
--------------------------------------------------


### Teacher Forcing

* Technique where the target word is passed as the next input to the decoder 


t-1번째에서 정확한 예측이 이루어졌다면 상관없지만,

잘못된 예측이 이루어졌다면 t번째 디코더의 추론 역시 잘못된 예측으로 이어질 것이다.

이전 예측을 고려해주는 디코더의 장점이 

잘못된 예측 앞에서는 엄청난 단점이 되어버린다.

특히 이러한 단점은 학습 초기에 학습 속도 저하의 요인이 된다.

이러한 단점을 해결하기 위해 나온 기법이 티쳐포싱(Teacher Forcing) 기법이다.

장 단점
장점: 학습 빠름
단점: 노출 편향 문제

# **Final Result and Insights**

# **References**

[??] Han, Lifeng. "Machine translation evaluation resources and methods: A survey." arXiv preprint arXiv:1605.04515 (2016).